In [1]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
import tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests

store_list    = ["ricardoeletro", "magazineluiza"]

dataset_pages = f"data/heuristic_pages/*/*"
dataset_links = f"data/heusristic_links/heusristic_links/*"

In [4]:
dataset_data = [(url, pathlib.Path(url).stem.split('_')[0] ) for url in glob.glob(dataset_links) if len(url.split('_'))>3]

In [5]:
datasets = []

dataset_idx = 0
data_raw_s = pd.read_csv(dataset_data[dataset_idx][0])
data_raw_s = data_raw_s.dropna().drop_duplicates()
data_raw_s['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw_s.shape[0])

for dataset_idx in range(1, len(dataset_data)):
    data_raw = pd.read_csv(dataset_data[dataset_idx][0])
    data_raw = data_raw.dropna().drop_duplicates()
    data_raw['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw.shape[0])
    data_raw_s = pd.merge(data_raw_s, data_raw, how='outer')
    #datasets.append(data_raw)

In [58]:
x_pages = [f'data/heuristic_pages/{store}/page_{p_id}.html' for (store, p_id) in zip(data_raw_s['store'], data_raw_s['id'])]
y = data_raw_s['label'].values
u_class = data_raw_s['store']
l = data_raw_s['links'].values

In [22]:
X, idx_arr = pages_to_matrix(x_pages)
y = y[idx_arr]
u_class = u_class[idx_arr]
l = l[idx_arr]
X.shape, y.shape, u_class.shape, l.shape


Invalid Links 4


In [63]:
df = pd.DataFrame()
df['store'] = u_class
df['link'] = l
df['text'] = X
df['label'] = y
df.to_csv("text_store_label.csv", index=False)

### Test

In [136]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

In [113]:
from sklearn.naive_bayes import GaussianNB
clss = [GaussianNB(), MultinomialNB(), KNeighborsClassifier(), SVC(gamma='auto'), 
        RandomForestClassifier(n_estimators=200), GradientBoostingClassifier(n_estimators=200)]

In [114]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [132]:
CV = CountVectorizer()
    
for cls in clss:

    X_vec = CV.fit_transform(X_train).toarray()
    X_vec_test = CV.transform(X_test).toarray()
    
    cls.fit(X_vec, y_train)  
    predicted = cls.predict(X_vec_test)
    print(cls.__class__.__name__, np.mean(predicted == y_test))
    

GaussianNB 0.767097966728281
MultinomialNB 0.9537892791127541
KNeighborsClassifier 0.9759704251386322
SVC 0.9593345656192237
RandomForestClassifier 0.9796672828096118
GradientBoostingClassifier 0.9815157116451017


In [137]:

class DenseTransformer(MinMaxScaler):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [ ]:
CV = CountVectorizer()
    
for cls in clss:
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('tranf', DenseTransformer()),
        ('clf', cls),
    ])
    
    
    text_clf.fit(X_train, y_train)  
    predicted = text_clf.predict(X_test)
    
    
    print(cls.__class__.__name__, np.mean(predicted == y_test))
    

GaussianNB 0.7689463955637708
MultinomialNB 0.9630314232902033
KNeighborsClassifier 0.9685767097966729
SVC 0.7652495378927912


In [131]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

text_clf.fit(X_train, y_train)  
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.9630314232902033